<a href="https://colab.research.google.com/github/lsteffenel/CHPS0906/blob/main/TP1/2-Tensors%20avec%20PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Introduction au Deep Learning avec PyTorch

Dans ce notebook vous découvrirez [PyTorch](http://pytorch.org/), un framework pour la création et l'entraînement de réseaux neuronaux. PyTorch prend prend en charge la gestion des tenseurs et simplifie leur déplacement vers les GPU pour le traitement plus rapide nécessaire lors de l'entraînement des réseaux neuronaux. Il fournit également un module qui calcule automatiquement les gradients (pour la rétropropagation !) et un autre module spécifiquement destiné à la création de réseaux neuronaux.



## Réseaux neuronaux

Le Deep Learning est basé sur des réseaux de neurones artificiels qui existent sous une forme ou une autre depuis la fin des années 1950. Les réseaux sont construits à partir de parties individuelles se rapprochant du comportement des neurones, généralement appelées unités ou simplement « neurones ». Chaque unité possède un certain nombre d'entrées pondérées par des poids *w*. Ces entrées pondérées sont additionnées (une combinaison linéaire) puis passées à travers une fonction d'activation pour obtenir la sortie de l'unité.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/simple_neuron.png?raw=1" width=400px>

Mathématiquement, cela ressemble à :

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

Avec les vecteurs, il s'agit du produit scalaire :

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensors

Il s'avère que les calculs de réseaux de neurones ne sont qu'un ensemble d'opérations d'algèbre linéaire sur des *tensors*, une généralisation des matrices. Un vecteur est un tensor unidimensionnel, une matrice est un tensor bidimensionnel, un tableau à trois indices est un tensor tridimensionnel (images couleur RGB par exemple). La structure de données fondamentale des réseaux de neurones est constituée de tensors et PyTorch (ainsi que pratiquement tous les autres frameworks d'apprentissage profond) est construit autour de tensors.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/tensor_examples.svg?raw=1" width=600px>

Une fois les bases abordées, il est temps d'explorer comment nous pouvons utiliser PyTorch pour créer un réseau  simple.

In [1]:
# First, import PyTorch
import torch

In [2]:
def activation(x):
    """ Sigmoid activation function

        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))
    # on voit ici que la fonction Sigmoid utilise une exponentiation, ce qui reste une opération couteuse

In [3]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

Ci-dessus, nous avons généré des données que nous pouvons utiliser pour obtenir la sortie de notre réseau simple. Tout cela est aléatoire pour l'instant, à l'avenir, nous commencerons à utiliser des données réélles. En parcourant chaque ligne pertinente, on trouve :

`features = torch.randn((1, 5))` crée un tensor de forme `(1, 5)`, i.e., une ligne et cinq colonnes, qui contient des valeurs distribuées aléatoirement selon la distribution normale avec une moyenne de zéro et un écart type de un.

`weights = torch.randn_like(features)` crée un autre tensor de même forme que `features`, contenant à nouveau des valeurs d'une distribution normale.

Enfin, `bias = torch.randn((1, 1))` crée une valeur unique à partir d'une distribution normale.

Les tensors PyTorch peuvent être ajoutés, multipliés, soustraits, etc. En général, vous utiliserez les tensors PyTorch à peu près de la même manière que vous utiliseriez les tableaux Numpy. Ils présentent cependant quelques avantages intéressants, comme l'accélération GPU que nous aborderons plus tard. Pour l'instant, utilisez les données générées pour calculer la sortie de ce réseau simple à une seule couche.


> **Exercice** : Calculez la sortie du réseau avec les fonctionnalités d'entrée `features`, les poids `weights` et le biais `bias`. Similairement à Numpy, PyTorch dispose d'une fonction [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum), ainsi que d'une méthode `.sum()` sur les tensors, pour prendre des sommes. Utilisez la fonction `activation` définie ci-dessus comme fonction d'activation.

In [ ]:
## Calculez la sortie de ce réseau en utilisant les poids et les tensors de biais

Vous pouvez effectuer la multiplication et la somme dans la même opération en utilisant une multiplication matricielle. En général, vous souhaiterez utiliser des multiplications matricielles car elles sont plus efficaces et accélérées à l'aide de bibliothèques modernes et de calculs sur les GPU.

Ici, nous voulons effectuer une multiplication matricielle des caractéristiques et des poids. Pour cela, nous pouvons utiliser [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) ou [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul) qui est un peu plus compliqué et prend en charge la diffusion. Si nous essayons de le faire avec `features` et `weights` tels quels, nous obtiendrons une erreur

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

Lorsque vous construisez des réseaux neuronaux dans n'importe quel framework, Vous verrez cela souvent. **Très souvent**. Ce qui se passe ici, c'est que nos tenseurs n'ont pas les formes correctes pour effectuer une multiplication matricielle. Rappelez-vous que pour les multiplications matricielles, le nombre de colonnes du premier tensor doit être égal au nombre de lignes de la deuxième colonne. Les `features` et `weights` ont la même forme, `(1, 5)`. Cela signifie que nous devons changer la forme de `weights` pour que la multiplication matricielle fonctionne.

**Remarque :** pour voir la forme d'un tensor appelé `tensor`, utilisez `tensor.shape`. Si vous construisez des réseaux neuronaux, vous utiliserez souvent cette méthode.

Il y a quelques options pour adapter la forme des tensors : [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), et [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` renverra un nouveau tensor avec les mêmes données que `weights` avec une taille `(a, b)` parfois, et parfois un clone, comme s'il copiait les données dans une autre partie de la mémoire.
* `weights.resize_(a, b)` renvoie le même tensor avec une forme différente. Cependant, si la nouvelle forme produit moins d'éléments que le tensor d'origine, certains éléments seront supprimés du tensor (mais pas de la mémoire). Si la nouvelle forme produit plus d'éléments que le tensor d'origine, les nouveaux éléments ne seront pas initialisés en mémoire. Je dois ici noter que le trait de soulignement à la fin de la méthode indique que cette méthode est exécutée **sur place** (rappelez-vous des "inplace=True" dans Pandas ?). Voici un excellent fil de discussion sur le forum pour [en savoir plus sur les opérations sur place](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) dans PyTorch.
* `weights.view(a, b)` renverra un nouveau tensor avec les mêmes données que `weights` avec la taille `(a, b)`.

Nous allons utiliser `.view()`, mais n'importe laquelle des trois méthodes fonctionnera pour cela. Nous pouvons donc maintenant remodeler `weights` pour avoir cinq lignes et une colonne avec quelque chose comme `weights.view(5, 1)`.

> **Exercice** : Calculez la sortie de notre petit réseau en utilisant la multiplication matricielle.

In [ ]:
## Calculez la sortie de ce réseau en utilisant la multiplication matricielle

### Empilez-les !

Voilà comment vous pouvez calculer la sortie d'un seul neurone. La véritable puissance de cet algorithme se produit lorsque vous commencez à empiler ces unités individuelles en couches et en piles de couches, dans un réseau de neurones. La sortie d'une couche de neurones devient l'entrée de la couche suivante. Avec plusieurs unités d'entrée et de sortie, nous devons maintenant exprimer les poids sous forme de matrice.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/multilayer_diagram_weights.png?raw=1' width=450px>

La première couche affichée en bas ici correspond aux entrées, appelées à juste titre la **couche d'entrée**. La couche intermédiaire est appelée la **couche cachée**, et la dernière couche (à droite) est la **couche de sortie**. Nous pouvons à nouveau exprimer ce réseau mathématiquement avec des matrices et utiliser la multiplication de matrices pour obtenir des combinaisons linéaires pour chaque unité en une seule opération. Par exemple, la couche cachée ($h_1$ et $h_2$ ici) peut être calculée

$$
\vec{h} = [h_1 \, h_2] =
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

La sortie de ce petit réseau est trouvée en traitant la couche cachée comme des entrées pour l'unité de sortie. La sortie du réseau est exprimée simplement

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [ ]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

> **Exercice :** calculez la sortie de ce réseau multicouche en utilisant les poids « W1 » et « W2 » et les biais « B1 » et « B2 ».

In [ ]:
## Votre solution ici

Si vous avez fait cela correctement, vous devriez voir le résultat `tensor([[ 0.3171]])`.

Le nombre d'unités cachées est un paramètre du réseau, souvent appelé un **hyperparamètre** pour le différencier des paramètres de pondération et de biais. Comme vous le verrez plus tard lorsque nous aborderons la formation d'un réseau neuronal, plus un réseau comporte d'unités cachées et de couches, plus il est capable d'apprendre à partir des données et de faire des prédictions précises.

## Numpy vers Torch et vice-versa

Section bonus spéciale ! PyTorch dispose d'une fonctionnalité intéressante pour la conversion entre les tableaux Numpy et les tensors Torch. Pour créer un tensor à partir d'un tableau Numpy, utilisez `torch.from_numpy()`. Pour convertir un tensor en tableau Numpy, utilisez la méthode `.numpy()`.

In [ ]:
import numpy as np
np.set_printoptions(precision=8)
a = np.random.rand(4,3)
a

In [ ]:
torch.set_printoptions(precision=8)
b = torch.from_numpy(a)
b

In [ ]:
b.numpy()

La mémoire est partagée entre le tableau Numpy et le tensor Torch, donc si vous modifiez les valeurs à la place d'un objet, l'autre changera également.

In [ ]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

In [ ]:
# Numpy array matches new values from Tensor
a